# Solving Complex Tasks with A Sequence of Nested Chats

This notebook shows how you can leverage **nested chats** to solve complex task with AutoGen. Nested chats is a sequence of chats created by a receiver agent after receiving a message from a sender agent and finished before the receiver agent replies to this message. Nested chats allow AutoGen agents to use other agents as their inner monologue to accomplish tasks. This abstraction is powerful as it allows you to compose agents in rich ways. This notebook shows how you can nest a pretty complex sequence of chats among _inner_ agents inside an _outer_ agent.


In [ ]:
import autogen

config_list_openai = [
    {
        'base_url': 'http://aitools.cs.vt.edu:7860/openai/v1',
        'api_key': 'aitools',
        'model': 'gpt-4-turbo-preview',
    }
]

llm_config_openai = {
    "timeout": 300,
    "seed": 42,
    "config_list": config_list_openai,
    "temperature": 0.1,
    "allow_format_str_template": True
}


### Example Task

Suppose we want the agents to complete the following sequence of tasks:

In [ ]:
tasks = [
    """Write a Python class to implement a concurrent skip list.""",
    """Make a pleasant joke about it.""",
]

Since the first task could be complex to solve, lets construct new agents that can serve as an inner software developer monologue.

### Step 1. Define Agents

#### A Group Chat for Developer Monologue
Below, we construct a group chat manager which manages an `developer_assistant` agent and an `developer_code_interpreter` agent. 
Later we will use this group chat inside another agent.


In [ ]:
developer_inner_assistant = autogen.AssistantAgent(
    "Developer-assistant",
    llm_config=llm_config_openai,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

developer_inner_code_interpreter = autogen.UserProxyAgent(
    "Developer-code-interpreter",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

developer_inner_groupchat = autogen.GroupChat(
    agents=[developer_inner_assistant, developer_inner_code_interpreter],
    messages=[],
    speaker_selection_method="round_robin",  # With two agents, this is equivalent to a 1:1 conversation.
    allow_repeat_speaker=False,
    max_round=8,
)

developer_manager = autogen.GroupChatManager(
    groupchat=developer_inner_groupchat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config_openai,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
)

#### Inner- and Outer-Level Individual Agents

Now we will construct a number of individual agents that will assume role of outer and inner agents.

In [ ]:
developer = autogen.AssistantAgent(
    name="Developer",
    llm_config={"config_list": config_list_openai},
)

snarchitect = autogen.AssistantAgent(
    name="Snarchitect",
    llm_config={"config_list": config_list_openai},
)

developer_commenter = autogen.AssistantAgent(
    name="Developer-commenter",
    llm_config={"config_list": config_list_openai},
    system_message="""
    You are a professional Python comment writer and typing signature author.
    Given some python code, ensure all docstring comments are completed, and all methods have type signatures.
    You transform Python code into commented and typed Python code.
    """,
)

developer_error_handler = autogen.AssistantAgent(
    name="Developer-Error-Handler",
    llm_config={"config_list": config_list_openai},
    system_message="""
    You are a professional Python coder, known for your thoroughness and commitment to 
    catching, handling and documenting exceptions from locations where developers forgot to handle them.
    Your task is to scrutinize Python for any harmful locations where exceptions are not handled and handle them.
    You transform Python code into Python code with world class exception handling.
    """,
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    }
)

### Step 2: Orchestrate Nested Chats to Solve Tasks

#### Outer Level
In the following code block, at the outer level, we have communication between:

- `user` - `developer` for solving the first task, i.e., `tasks[0]`.
- `user` - `snarchitect` for solving the second task, i.e., `tasks[1]`.

#### Inner Level (Nested Chats)
Since the first task is quite complicated, we created a sequence of _nested chats_ as the inner monologue of Developer.

1. `developer` - `developer_manager`: This chat intends to delegate the task received by developer to the Developer Manager to solve.

2. `developer` - `developer_commenter`: This chat takes the output from Nested Chat 1, i.e., Developer vs. Developer Manager, and lets the Commenter polish the content to make a nice commented piece of code.

3. `developer` - `developer_error_handler`: This chat takes the output from Nested Chat 2 and intends to let the Reviewer agent review the content from Nested Chat 2.

4. `developer` - `developer_commenter`: This chat takes the output from previous nested chats and intends to let the Commenter agent finalize the code.

The sequence of nested chats can be realized with the `register_nested_chats` function, which allows one to register one or a sequence of chats to a particular agent (in this example, the `developer` agent).

Information about the sequence of chats can be specified in the `chat_queue` argument of the `register_nested_chats` function. The following fields are especially useful:
- `recipient` (required) specifies the nested agent;
- `message` specifies what message to send to the nested recipient agent. In a sequence of nested chats, if the `message` field is not specified, we will use the last message the registering agent received as the initial message in the first chat and will skip any subsequent chat in the queue that does not have the `message` field. You can either provide a string or define a callable that returns a string.
- `summary_method` decides what to get out of the nested chat. You can either select from existing options including `"last_msg"` and `"reflection_with_llm"`, or or define your own way on what to get from the nested chat with a Callable.
- `max_turns` determines how many turns of conversation to have between the concerned agent pairs.

In [ ]:
def writing_message(recipient, messages, sender, config):
    return f"Polish the code to make engaging and nicely formatted Python code. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"


nested_chat_queue = [
    {"recipient": developer_manager, "summary_method": "reflection_with_llm"},
    {"recipient": developer_commenter, "message": writing_message, "summary_method": "last_msg", "max_turns": 1},
    {"recipient": developer_error_handler, "message": "Review the content provided.", "summary_method": "last_msg", "max_turns": 1},
    {"recipient": developer_commenter, "message": writing_message, "summary_method": "last_msg", "max_turns": 1},
]
developer.register_nested_chats(
    nested_chat_queue,
    trigger=user,
)
# user.initiate_chat(assistant, message=tasks[0], max_turns=1)

res = user.initiate_chats(
    [
        {"recipient": developer, "message": tasks[0], "max_turns": 1, "summary_method": "last_msg"},
        {"recipient": snarchitect, "message": tasks[1]},
    ]
)